# Rvalue references and movable objects

## Reminders about classic references

#### A reference is an additional name for another pre-existing variable

In [ ]:
{
  int var = 42 ;
  int & ref = var ; // ref and var are the same
  ref = 99 ;        // ref and var are assigned
  assert(var==99) ; // 
}

Above, the variable `var` is said to be a **left value** (lvalue), because it can be placed on the left of `=` in an assignment instruction, because it has an associated slot in memory, where to store the result of the expression on the right of `=`.

A value which is somehow temporary, and cannot be on the left of a `=`, is said to be a **right value** (rvalue).

####  A reference cannot refer to a rvalue...

In [ ]:
{
  int & i = 42 ;   // Won’t compile
}

In [ ]:
#include <iostream>
#include <string>
void print( std::string & s )
 { std::cout<<s<<std::endl ; }

In [ ]:
{
  print("hello") ; // Won't compile
}

#### ... unless promising not to change it

In [ ]:
{
    int const & i = 42 ; // OK
}

In [ ]:
#include <iostream>
#include <string>
void print_const( std::string const & s )
 { std::cout<<s<<std::endl ; }

In [ ]:
{
    print_const("hello world") ; // OK
}

## Rvalue references (`&&`)

#### A "rvalue reference" can only refer to a rvalue

In [ ]:
{
    int && i = 42 ;
    int j = 42 ;
    int && k = j ; // Won’t compile
}

#### One can overload a function for rvalue references

In [ ]:
#include <iostream>
void f( int & )       { std::cout<<"(      &) "<<std::endl ; }

In [ ]:
void f( int const & ) { std::cout<<"(const &) "<<std::endl ; }

In [ ]:
void f( int && )      { std::cout<<"(     &&) "<<std::endl ; }

In [ ]:
{
    int i = 42 ;
    int const & j = i ;
    f(i) ;
    f(j) ;
    f(42) ;
}

## An opportunity for optimisation

#### The function below sort and display a collection of integers

In [ ]:
#include <iostream>
#include <vector>
void process( std::vector<int> const & col )
{
    std::vector<int> copy(col) ;
    std::sort(copy.begin(),copy.end()) ;
    std::cout<<"(const &) " ;
    for ( int element : copy )
     { std::cout<<element<<" " ; }
    std::cout<<std::endl ;
}

In [ ]:
#include <vector>
{
    const std::vector<int> v1 {1,3,2} ;
    std::vector<int> v2 {6,4,5} ;
    process(v1) ;
    process(v2) ;
    process({9,8,7}) ;
}

#### The overload below avoid the copy, but do not accept the rvalue

In [ ]:
void process( std::vector<int> & col )
{
    std::sort(col.begin(),col.end()) ;
    std::cout<<"(      &) " ;
    for ( int element : col )
     { std::cout<<element<<" " ; }
    std::cout<<std::endl ;
}

In [ ]:
{
    const std::vector<int> v1 {1,3,2} ;
    std::vector<int> v2 {6,4,5} ;
    process(v1) ;
    process(v2) ;
    process({9,8,7}) ;
}

#### The new overoad below avoid the copy for the rvalue

In [ ]:
void process( std::vector<int> && col )
{
    std::sort(col.begin(),col.end()) ;
    std::cout<<"(     &&) " ;
    for ( int element : col )
     { std::cout<<element<<" " ; }
    std::cout<<std::endl ;
}

In [ ]:
{
    std::vector<int> const v1 {1,3,2} ;
    std::vector<int> v2 {6,4,5} ;
    process(v1) ;
    process(v2) ;
    process({9,8,7}) ;
}

## When a function call another function

#### A rvalue reference is not an rvalue

If a variable is an rvalue reference, beginners generally assume that it is itself some kind of rvalue, and will be considered as so when transmitted to another function. **IT IS NOT**.

In the code below, `col` refers to a rvalue (`{9,8,7}`), but `col` itself is not a temporary value, as testified by the call to `process()`, which the compiler interpret as `process( std::vector<int> & )`.

In [ ]:
void super_process( std::vector<int> && col )
 { process(col) ; }

In [ ]:
{
  super_process({9,8,7}) ;
}

#### Transform any value into an rvalue with `std::move`

When you want a value, any, to be considered as an rvalue, you can simply call `std::move` on it. This is a way to say that the value is somehow "temporary", "movable", soon destructed, and can be directly reused and modified as needed.

In [ ]:
void super_process_2( std::vector<int> && col )
 { process(std::move(col)) ; }

In [ ]:
{
  super_process_2({9,8,7}) ;
}

## What about classes ?

<!-- mes tentatives de faire des classes simples de démonstration ont échouées à cause des capacités du compilateur à optimiser le code inutile. C'est pourquoi je me rabat ci-dessous sur des exemples non compilés, juste pour exposer les principes -->

#### When a class manage dynamic resources

If a class include a huge array of `int`, and if the input is a temporary object which will be thrown away afterwards, it makes no sense to "deep copy" all those values. To avoid such a useless deep copy, we can provide what is called a "move constructor" :

```c++
class A
 {
  private :
    int * m_data ;
  public :
    A() : m_data(new int[1000000]) {}
    ~A() { delete [] m_data; }
    A( A const & other )
     : m_data(new int[1000000])
     { std::copy(other.m_data,other.m_data+1000000,m_data) ; }
    A( A && other )
     : m_data(other.m_data)
     { other.m_data = nullptr ; }
 } ;
```

#### Actually, it is now recomanded to define "The Big Five"

```c++
A( A const & )             ; // copy constructor
A( A && )                  ; // move constructor
A & operator=( A const & ) ; // copy assignment
A & operator=( A && )      ; // move assignment
~A()                       ; // destructor
```

#### In the standard library

There are standard classes which are not copyable (the copy constructor is `= delete`), generally because they handle a resource which must not be duplicated. Yet they are movable:
* `std::unique_ptr`, which can be used within a `std::vector`, on the contrary of the obsolete `std::auto_ptr` ;
* `std::thread` ;
* etc.

## Exercise

The class below is some kind of simplified `std::string`.
* Add a move constructor and a move assignment operator. Check that `main()` use the move constructor.
* Add some display here and there, or use debugger, so to check when the move constructor is called... what do you think ?

In [2]:
%%file tmp.move.cpp

#include <cstring>
#include <iostream>

class Text
 {
  public :
    
    Text()
     : m_size(0),
       m_data(0)
     {}
    
    Text( char const * str )
     : m_size(std::strlen(str)),
       m_data(m_size?(new char [m_size+1]):0)
     { if (m_size) std::copy(str,str+m_size+1,m_data) ; }
    
    Text( Text const & t )
     : m_size(t.m_size),
       m_data(m_size?(new char [m_size+1]):0)
     {
      std::cout<<"copy constructor"<<std::endl ;
      if (m_size) std::copy(t.m_data,t.m_data+m_size+1,m_data) ;
     }
    
    Text & operator=( Text const & t )
     {
      std::cout<<"copy assignment"<<std::endl ;
      if (this == &t) return *this ;
      delete [] m_data ;
      m_size = t.m_size ;
      m_data = m_size?(new char [m_size+1]):0 ;
      if (m_size) std::copy(t.m_data,t.m_data+m_size+1,m_data) ;
      return *this ;
     }
    
    ~Text()
     { delete [] m_data ; }
    
    unsigned int taille()
     { return m_size ; }
    
    char & operator[]( unsigned int i )
     { return m_data[i] ; }
    
    friend std::ostream & operator<<( std::ostream & os, Text const & t )
     { if (t.m_size) return os<<t.m_data ; else return os ; }
    
  private :
    
    unsigned int m_size ;
    char * m_data ;
 } ;
 
Text hello()
 { return "hello" ; }

Text uppercase( Text t )
 {
  for ( unsigned int i=0 ; i<t.taille() ; ++i )
   { t[i] = std::toupper(t[i]) ; }
  return t ;
 }

int main()
 {
  std::cout<<uppercase(hello())<<std::endl ;
  return 0 ;
 }

Overwriting tmp.move.cpp


In [3]:
!rm -f tmp.move.exe && g++ -std=c++17 tmp.move.cpp -o tmp.move.exe

In [4]:
!./tmp.move.exe

copy constructor
HELLO


© *CNRS 2020*  
*Assembled and written by David Chamont, this work is made available according to the terms of the*  
[*Creative Commons License - Attribution - NonCommercial - ShareAlike 4.0 International*](http://creativecommons.org/licenses/by-nc-sa/4.0/)